In [1]:
import os
import glob
import cv2

from time import sleep

import torch
import torchvision

import matplotlib.pyplot as plt
%matplotlib inline

from dataset import WeizmannHumanActionVideo
from image_autoencoder import ImageAutoEncoder
from lstm_autoencoder import VideoAutoEncoder

In [2]:
#  use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Dataset 

In [3]:
"""
trans_data = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
"""

trans_data = torchvision.transforms.ToTensor()
trans_label = None

dataset = WeizmannHumanActionVideo(trans_data=None, trans_label=trans_label, train=True)

## Train-test split

In [4]:
train_size = int(1.0 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [5]:
print("train: ", len(train_dataset))
print("test: ", len(test_dataset))

train:  93
test:  0


## Training

**Dataloader**

In [6]:
batch_size=1

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True, 
                                           num_workers=4)

"""
test_loader = torch.utils.data.DataLoader(test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True, 
                                           num_workers=1)
"""

'\ntest_loader = torch.utils.data.DataLoader(test_dataset, \n                                           batch_size=batch_size, \n                                           shuffle=True, \n                                           num_workers=1)\n'

In [8]:
# type(data)

**Iterative algorithm (SGD)**

In [9]:
n_epochs=10

In [10]:
model = VideoAutoEncoder(input_size=16, hidden_size=16).to(device)

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

criterion = torch.nn.BCELoss()

In [13]:
for epoch in range(n_epochs):
    model.train()
    train_loss = 0
    
    for batch_id, (batch_data, _) in enumerate(train_loader):
        # print(torch.cuda.memory_allocated(device))
        
        # batch_data: 5D-Tensor (batch_size=1, video_len, channel, height, width)
        x = batch_data.to(device) 
               
        optimizer.zero_grad()
    
        # x_hat: (video_len, channel, height, width)
        # zc: (video_len, dim_zc)        
        x_hat = model(x) 

        loss = criterion(x_hat, x)
        loss.backward() # compute accumulated gradients
        
        train_loss += loss.item()

        optimizer.step()
                
        print("epoch : {}/{}, batch : {}/{}, loss = {:.6f}".format(
            epoch + 1, n_epochs, batch_id, int(len(train_dataset)/batch_size), loss.item()))   
        
        del x_hat
        
    print("epoch : {}/{}, loss = {:.4f}".format(epoch + 1, n_epochs, train_loss / len(train_loader)))

epoch : 1/10, batch : 0/93, loss = 0.681001
epoch : 1/10, batch : 1/93, loss = 0.680791
epoch : 1/10, batch : 2/93, loss = 0.701332
epoch : 1/10, batch : 3/93, loss = 0.684154
epoch : 1/10, batch : 4/93, loss = 0.684569
epoch : 1/10, batch : 5/93, loss = 0.675087
epoch : 1/10, batch : 6/93, loss = 0.680765
epoch : 1/10, batch : 7/93, loss = 0.677295
epoch : 1/10, batch : 8/93, loss = 0.674516
epoch : 1/10, batch : 9/93, loss = 0.670695
epoch : 1/10, batch : 10/93, loss = 0.675676
epoch : 1/10, batch : 11/93, loss = 0.674949
epoch : 1/10, batch : 12/93, loss = 0.677071
epoch : 1/10, batch : 13/93, loss = 0.666294
epoch : 1/10, batch : 14/93, loss = 0.681915
epoch : 1/10, batch : 15/93, loss = 0.684309
epoch : 1/10, batch : 16/93, loss = 0.664549
epoch : 1/10, batch : 17/93, loss = 0.672653
epoch : 1/10, batch : 18/93, loss = 0.661864
epoch : 1/10, batch : 19/93, loss = 0.670607
epoch : 1/10, batch : 20/93, loss = 0.670510
epoch : 1/10, batch : 21/93, loss = 0.664692
epoch : 1/10, batch 

KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, b